In [25]:
import torch
import torch.nn as nn

from lib.Exercise1_1 import LQRSolver
from torch.utils.data import TensorDataset, DataLoader
class DGMNN(nn.Module):
    def __init__(self):
        super(DGMNN, self).__init__()
        self.layer1 = nn.Linear(3, 100)  
        self.layer2 = nn.Linear(100, 100)
        self.layer3 = nn.Linear(100, 100)
        self.tanh = nn.Tanh()
        self.output = nn.Linear(100, 1)
        
    def forward(self, x):
        x = self.tanh(self.layer1(x))
        x = self.tanh(self.layer2(x))
        x = self.tanh(self.layer3(x))
        return self.output(x)



In [26]:
def get_hessian(grad,x):
    Hessian = torch.tensor([])
    
    for i in range(len(x)):
        hessian = torch.tensor([])
        for j in range(len(grad[i])):
            u_xxi = torch.autograd.grad(grad[i][j], x, grad_outputs=torch.ones_like(grad[i][j]), retain_graph=True,create_graph=True, allow_unused=True)[0]           
            hessian = torch.cat((hessian, u_xxi[i].unsqueeze(0)))
        Hessian = torch.cat((Hessian, hessian.unsqueeze(0)),dim = 0)
    return Hessian

def pde_residual(model, t, x):
    
    input = torch.cat((t.unsqueeze(1), x),dim=1)
    
    u = model(input)

    u_t = torch.autograd.grad(u, t, grad_outputs=torch.ones_like(u), create_graph=True, retain_graph=True)[0]
    u_x = torch.autograd.grad(u, x, grad_outputs=torch.ones_like(u), create_graph=True, retain_graph=True)[0]

    u_xx = get_hessian(u_x,x)

    residual = u_t + 0.5 * torch.einsum('bii->b', sigma @ sigma.transpose(1,2) @ u_xx) + (u_x.unsqueeze(1) @ (H @ x.unsqueeze(1).transpose(1,2)) + u_x.unsqueeze(1) @ M @ alpha + x.unsqueeze(1) @ C @ x.unsqueeze(1).transpose(1,2) + alpha.transpose(1,2) @ D @ alpha).squeeze()
    return residual

def boundary_condition(model,t, x):
    # 定义边界条件的残差
    
    T_input = T * torch.ones_like(t)

    input = torch.cat((T_input.unsqueeze(1), x),dim=1)
    u = model(input)

    return u - (x.unsqueeze(1) @ R @ x.unsqueeze(1).transpose(1,2)).squeeze()

def total_residual(model, t, x):
    # 计算内部PDE残差
    residual_loss = pde_residual(model, t, x).pow(2).mean()
    # 计算边界条件的残差
    boundary_loss = boundary_condition(model,t,x).pow(2).mean()
    
    return residual_loss + boundary_loss


In [3]:
def new_data(num_samples):
    #num_samples = 10000
    t_samples = T * torch.rand(num_samples, dtype=torch.double, requires_grad=True)
    x_ends = torch.tensor([-3,3], dtype = torch.double)
    x_samples = x_ends[0] + (x_ends[1]- x_ends[0]) * torch.rand(num_samples , 2, dtype=torch.double, requires_grad=True)
    return t_samples,x_samples

In [27]:
# Define matrices for LQR problem
H = torch.tensor([[1.2, 0.8], [-0.6, 0.9]], dtype=torch.double)
M = torch.tensor([[0.5,0.7], [0.3,1.0]], dtype=torch.double)
sigma = torch.tensor([[[0.8],[1.1]]], dtype=torch.double)
alpha = torch.tensor([[[1],[1]]], dtype=torch.double)
C = torch.tensor([[1.6, 0.0], [0.0, 1.1]], dtype=torch.double)
D = torch.tensor([[0.5, 0.0], [0.0, 0.7]], dtype=torch.double)
R = torch.tensor([[0.9, 0.0], [0.0, 1.0]], dtype=torch.double)
T = torch.tensor(1.0, dtype=torch.double)

solver = LQRSolver(H, M, sigma, C, D, R, T=T, method="euler")


In [5]:
model_DGM = DGMNN().double()

# Prepare for training
optimizer_DGM = torch.optim.Adam(model_DGM.parameters(), lr=0.001)
epoch_losses = []

batch_size = 30
epochs = 500

for batch in range(batch_size):
    print(f'Batch {batch+1}/{batch_size}'+'\n')
    
    t_data,x_data = new_data(50)
    dataset = TensorDataset(t_data,x_data)
    dataloader = DataLoader(dataset, batch_size=128, shuffle=True)

    for epoch in range(epochs):

        model_DGM.train()
        total_loss = 0
        
        for batch_idx, (t_data_,x_data_) in enumerate(dataloader):
            optimizer_DGM.zero_grad()
            loss = total_residual(model_DGM, t_data_, x_data_) 
            loss.backward(retain_graph=True)
            optimizer_DGM.step()
            total_loss += loss.item()
        epoch_losses.append(total_loss / len(dataloader))

        if epoch == 0:
            print(f'Epoch {epoch+1}/{epochs} \t Loss: {total_loss / len(dataloader)}')
        if (epoch+1) % 100 == 0:
            print(f'Epoch {epoch+1}/{epochs} \t Loss: {total_loss / len(dataloader)}')

print('\n')
model_DGM.eval()

Batch 1/30

Epoch 1/500 	 Loss: 179.38920943757603
Epoch 100/500 	 Loss: 55.63965676485591
Epoch 200/500 	 Loss: 21.000615344534598
Epoch 300/500 	 Loss: 16.933412291149832
Epoch 400/500 	 Loss: 15.0249363441466
Epoch 500/500 	 Loss: 13.844563649177934
Batch 2/30

Epoch 1/500 	 Loss: 19.503522725018385
Epoch 100/500 	 Loss: 15.59205568977103
Epoch 200/500 	 Loss: 14.961941164535304
Epoch 300/500 	 Loss: 14.578325733578623
Epoch 400/500 	 Loss: 14.280125460164486
Epoch 500/500 	 Loss: 14.020513356986141
Batch 3/30

Epoch 1/500 	 Loss: 25.340556877811157
Epoch 100/500 	 Loss: 19.11800524720242
Epoch 200/500 	 Loss: 18.613381802375137
Epoch 300/500 	 Loss: 18.314198112088807
Epoch 400/500 	 Loss: 18.106350705380343
Epoch 500/500 	 Loss: 17.96623103348157
Batch 4/30

Epoch 1/500 	 Loss: 18.316024904658608
Epoch 100/500 	 Loss: 14.321167523864505
Epoch 200/500 	 Loss: 14.088331209657527
Epoch 300/500 	 Loss: 13.95257659499725
Epoch 400/500 	 Loss: 13.880596541372851
Epoch 500/500 	 Loss: 13

DGMNN(
  (layer1): Linear(in_features=3, out_features=50, bias=True)
  (layer2): Linear(in_features=50, out_features=50, bias=True)
  (tanh): Tanh()
  (output): Linear(in_features=50, out_features=1, bias=True)
)

In [19]:
# Initialization
t_data,x_data = new_data(5)

value_numerical = solver.value_function(t_data.detach(),x_data.detach().unsqueeze(1))

In [20]:
A = model_DGM(torch.cat((t_data.unsqueeze(1), x_data.squeeze(1)),dim=1))

In [23]:
A.squeeze()

tensor([13.8489,  7.5302,  8.5210,  7.5368, 14.1952], dtype=torch.float64,
       grad_fn=<SqueezeBackward0>)

In [24]:
value_numerical

tensor([39.2751, 27.3157,  9.1775, 18.6481, 10.6329], dtype=torch.float64)